In [25]:
# S3 접근하기
import boto3
s3_resource = boto3.resource('s3')

In [2]:
# Athena 사용하기
import boto3
client = boto3.client('athena')

In [1]:
# Athena에서 쿼리 실행 후 pandas dataframe으로 받기
from pyathena import connect
import pandas as pd
import config

aws_access_key = config.Access_key_ID
aws_secret_access_key = config.Scret_access_key
region = 'ap-northeast-2'
default_dir = 's3://adbrix.gh.data/'



conn = connect(aws_access_key_id=aws_access_key,
               aws_secret_access_key=aws_secret_access_key,
               s3_staging_dir=default_dir,
               region_name=region)
df = pd.read_sql("SELECT adid FROM sampledb.adtouch_table_201908 limit 100", conn)
print(df.head())

                               adid
0  d41d8cd98f00b204e9800998ecf8427e
1  d41d8cd98f00b204e9800998ecf8427e
2  d41d8cd98f00b204e9800998ecf8427e
3  d41d8cd98f00b204e9800998ecf8427e
4  d41d8cd98f00b204e9800998ecf8427e


In [20]:
# parameter 전달하기 기본형 --> 안됨. 걍 파이썬에서 string operation으로 해결하자
filtering_query = """
SELECT adid, 
       counts 
FROM 
    (SELECT adid, 
         count(*) counts 
    FROM %(param)s -- ITERATION: 월별 애드터치 데이터 
    
    GROUP BY  adid ) 

GROUP BY adid, counts 

HAVING counts <= APPROX_PERCENTILE(counts, 0.8)
"""

sample_query = "SELECT %(param)s FROM sampledb.adtouch_table_201908 limit 100"

cursor = connect(aws_access_key_id=aws_access_key,
                   aws_secret_access_key=aws_secret_access_key,
                   s3_staging_dir=default_dir,
                   region_name=region).cursor()

cursor.execute(sample_query, {'param': "adid"})

print(cursor.fetchall())

[('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',), ('adid',)

### pandas cursor  
PandasCursor directly handles the CSV file of the query execution result output to S3. This cursor is to download the CSV file after executing the query, and then loaded into DataFrame object. Performance is better than fetching data with a cursor.

In [ ]:
# pandas cursor 활용하기

from pyathena.pandas_cursor import PandasCursor

cursor = connect(aws_access_key_id=aws_access_key,
               aws_secret_access_key=aws_secret_access_key,
               s3_staging_dir=default_dir,
               region_name=region,
               cursor_class=PandasCursor).cursor()

In [2]:
# filtering adid from adtouch data

yyyymm = '201908'
percentile = '0.8'

filtering_query = """
SELECT adid, 
       counts 
FROM 
    (SELECT adid, 
         count(*) counts 
    FROM sampledb.adtouch_table_{0} -- ITERATION: 월별 애드터치 데이터 
    
    GROUP BY  adid ) 

GROUP BY adid, counts 

HAVING counts <= APPROX_PERCENTILE(counts, {1}) 
""".format(yyyymm, percentile)

df = cursor.execute(filtering_query).as_pandas()
print(df.head())

                               adid  counts
0  cefa46e8fe8b87d7c145f624661c2030       2
1  3f0d2144d617e77f9b89766db00f5ac5       1
2  1514d92de8c7ff28b7f3e86024db5cef       4
3  92597f93d5316cdb77ad1c37ebb75277      22
4  c12b43f32189cddfe472b8dfa2c9994f      33


In [7]:
# filtering purchase records of selected adid

purchase_query = """
-- 특정 카운트 이하로만 등장하는 adid 테이블
WITH filtered AS (
    SELECT adid, 
           counts 
    FROM 
        (SELECT adid, count(*) counts 
        FROM sampledb.adtouch_table_{0} -- ITERATION: 월별 애드터치 데이터 
        GROUP BY  adid ) 

    GROUP BY adid, counts 
    
    HAVING counts <= APPROX_PERCENTILE(counts, {1})
)


-- 해당 adid에 포함되는 adid만 purchase 뽑기
SELECT adid, server_datetime, evt_param_item_price * evt_param_item_quantity as amount

FROM sampledb.events_table_{0} t1 -- ITERATION: 월별 이벤트 데이터
WHERE event_name = 'abx:purchase'
and EXISTS (
  SELECT adid
  FROM filtered t2
  WHERE t1.adid = t2.adid
  );
""".format(yyyymm, percentile)

df = cursor.execute(purchase_query).as_pandas()
print(df.head())

                               adid         server_datetime    amount
0  ff51bbbb195cda4700013d27c0460dab 2019-08-26 14:40:29.931   63900.0
1  89b8c6a89871e7baab1fa3edf3e1fcde 2019-08-17 11:46:53.337  159000.0
2  c56cd0d8b27a7727e0c5186dc8207000 2019-08-21 04:45:08.462   14950.0
3  c56cd0d8b27a7727e0c5186dc8207000 2019-08-21 04:45:08.462   14950.0
4  c56cd0d8b27a7727e0c5186dc8207000 2019-08-21 04:45:08.462   14950.0


## 연습장

경원이 요청 union 뽑아주기

In [9]:
from query_handler import Handler
import config

curosr = Handler(config.Access_key_ID, config.Scret_access_key)

yyyymm = "201908"
percentile = "0.8"

union_query = """
-- 특정 카운트 이하로만 등장하는 adid 테이블
WITH filtered AS (
    SELECT adid, 
           counts 
    FROM 
        (SELECT adid, count(*) counts 
        FROM sampledb.adtouch_table_{0} -- ITERATION: 월별 애드터치 데이터 
        GROUP BY  adid ) 

    GROUP BY adid, counts 
    
    HAVING counts <= APPROX_PERCENTILE(counts, {1})
)


-- 해당 adid에 포함되는 adid만 purchase 뽑기
SELECT adid, server_datetime, evt_param_item_price * evt_param_item_quantity as amount

FROM sampledb.events_table_{0} t1 -- ITERATION: 월별 이벤트 데이터
WHERE event_name = 'abx:purchase'
and EXISTS (
  SELECT adid
  FROM filtered t2
  WHERE t1.adid = t2.adid
  );
""".format(yyyymm, percentile)

union_query

# df = cursor.execute(purchase_query).as_pandas()
# print(df.head())

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-6b72d6bc9e92>", line 1, in <module>
    from query_handler import Handler
ModuleNotFoundError: No module named 'query_handler'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2018, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/site-

ModuleNotFoundError: No module named 'query_handler'